In [ ]:
# project configurations

import sqlalchemy as sa
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import datetime
import os
import zipfile
import uuid
import matplotlib
import seaborn as sns
from os.path import basename
from datetime import datetime, date, time, timedelta


client_pg = sa.create_engine('postgresql://localhost:5432/postgres')

In [ ]:
all_trx = """
  select * from transactions order by created_date asc;
"""
df_all_trx = pd.read_sql(all_trx, client_pg)

df_all_trx.head()

In [ ]:
# question number two part one

# fraudulent users data

# sum of transactions distinct fraudulent user has made
sum_distinct_number_of_trx_made_by_fraudulent_user = """
select distinct(user_id), count(index) AS count from transactions where user_id in (select fraudsters.user_id
from fraudsters) group by user_id order by count desc;
"""
df_sum_fraud = pd.read_sql(sum_distinct_number_of_trx_made_by_fraudulent_user, client_pg)
df_sum_fraud.plot.hist('count', title='ALL')
indexNames = df_sum_fraud[ df_sum_fraud['count'] > 200 ].index
df_sum_fraud.drop(indexNames, inplace=True)
df_sum_fraud.plot.hist('count', bins=200)

# sum of declined transactions distinct fraudulent user has made
sum_distinct_number_of_trx_made_by_fraudulent_user = """
select distinct(user_id), count(index) AS count from transactions where user_id in (select fraudsters.user_id
from fraudsters) and state='DECLINED' group by user_id order by count desc;
"""
df_sum_fraud = pd.read_sql(sum_distinct_number_of_trx_made_by_fraudulent_user, client_pg)
df_sum_fraud.plot.hist('count', title='DECLINED')

# sum of completed transactions distinct fraudulent user has made
sum_distinct_number_of_trx_made_by_fraudulent_user = """
select distinct(user_id), count(index) AS count from transactions where user_id in (select fraudsters.user_id
from fraudsters) and state='COMPLETED' group by user_id order by count desc;
# """
df_sum_fraud = pd.read_sqil(sum_distinct_number_of_trx_made_by_fraudulent_user, client_pg)
df_sum_fraud.plot.hist('count', ttle='COMPLETED')

# normal users data

# sum of transactions distinct normal user has made
sum_distinct_number_of_trx_made_by_normal_user = """
select distinct(user_id), count(index) AS count from transactions where user_id not in (select fraudsters.user_id
from fraudsters)  group by user_id order by count desc;
"""
df_sum_normal = pd.read_sql(sum_distinct_number_of_trx_made_by_normal_user, client_pg)
indexNames = df_sum_normal[ df_sum_normal['count'] > 500 ].index
df_sum_normal.drop(indexNames, inplace=True)
df_sum_normal.plot.hist('count', title='ALL')

# sum of declined transactions distinct normal user has made
sum_distinct_number_of_trx_made_by_normal_user = """
select distinct(user_id), count(index) AS count from transactions where user_id not in (select fraudsters.user_id
from fraudsters) and state='DECLINED' group by user_id order by count desc;
"""
df_sum_normal = pd.read_sql(sum_distinct_number_of_trx_made_by_normal_user, client_pg)
indexNames = df_sum_normal[ df_sum_normal['count'] > 500 ].index
df_sum_normal.drop(indexNames, inplace=True)
df_sum_normal.plot.hist('count', title='DECLINED')

# sum of completed transactions distinct normal user has made
sum_distinct_number_of_trx_made_by_normal_user = """
select distinct(user_id), count(index) AS count from transactions where user_id not in (select fraudsters.user_id
from fraudsters) and state='COMPLETED' group by user_id order by count desc;
"""
df_sum_normal = pd.read_sql(sum_distinct_number_of_trx_made_by_normal_user, client_pg)
df_sum_normal.plot.hist('count', title='COMPLETED')
indexNames = df_sum_normal[ df_sum_normal['count'] > 500 ].index
df_sum_normal.drop(indexNames, inplace=True)
df_sum_normal.plot.hist('count', title='COMPLETED')


## scratches

# sum of distinct entry_method of trx by fraudulent users
sum_distinct_entry_method = """
select distinct(entry_method) as entry_method, count(entry_method) as number_of_occurrence from transactions
where user_id in (select fraudsters.user_id
from fraudsters)
group by entry_method
order by number_of_occurrence desc;
"""
# df = pd.read_sql(sum_distinct_entry_method, client_pg)
# df = df.set_index('entry_method')

# sum of common types of payments of fraudulent transactions
sum_of_payments_types = """
select distinct(type) as type, count(type) as number_of_occurrence from transactions
where user_id in (select fraudsters.user_id
from fraudsters)
group by type
order by number_of_occurrence desc;
"""
# df = pd.read_sql(sum_of_payments_types, client_pg)
# df = df.set_index('type')
# plot = df.plot.pie(y='number_of_occurrence', figsize=(5, 5), title="number of occurence of individual type")

# sum of distinct entry_method of trx by fraudulent users
sum_distinct_entry_method = """
select distinct(entry_method) as entry_method, count(entry_method) as number_of_occurrence from transactions
where user_id in (select fraudsters.user_id
from fraudsters)
group by entry_method
order by number_of_occurrence desc;
"""
df = pd.read_sql(sum_distinct_entry_method, client_pg)
df = df.set_index('entry_method')
df.plot.pie(y='number_of_occurrence', figsize=(5, 5), title="number of occurence of individual type")

# sum of common types of payments of fraudulent transactions
sum_of_payments_types = """
select distinct(type) as type, count(type) as number_of_occurrence from transactions
where user_id in (select fraudsters.user_id
from fraudsters)
group by type
order by number_of_occurrence desc;
"""
df = pd.read_sql(sum_of_payments_types, client_pg)
df = df.set_index('type')
plot = df.plot.pie(y='number_of_occurrence', figsize=(5, 5), title="number of occurence of individual type")

In [ ]:
# majority distribution of fraudulent users trx
majority_distribution_of_fraudulent_users_trx = """
with fr_usr as (select distinct(user_id), count(index) AS count from transactions where user_id in (select fraudsters.user_id
from fraudsters) group by user_id order by count desc)
select user_id from fr_usr where count < 200;
"""
df_mj_fr_usr = pd.read_sql(majority_distribution_of_fraudulent_users_trx, client_pg)
fr_usr_ids = df_mj_fr_usr['user_id'].tolist()
df_trx_by_fr_usr = df_all_trx[df_all_trx.user_id.isin(fr_usr_ids)]

df_entry_method_by_fr_usr = df_trx_by_fr_usr.groupby('entry_method').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_entry_method_by_fr_usr.head()

# df_type_by_fr_usr = df_trx_by_fr_usr.groupby('type').size().reset_index(name='counts').sort_values('counts', ascending=False)
# df_type_by_fr_usr.head()

# df_currency_by_fr_usr = df_trx_by_fr_usr.groupby('currency').size().reset_index(name='counts').sort_values('counts', ascending=False)
# df_currency_by_fr_usr.head()

# df_merchant_country_by_fr_usr = df_trx_by_fr_usr.fillna('null').groupby('merchant_country').size().reset_index(name='counts').sort_values('counts', ascending=False)
# df_merchant_country_by_fr_usr.head()

# df_merchant_category_by_fr_usr = df_trx_by_fr_usr.fillna('null').groupby('merchant_category').size().reset_index(name='counts').sort_values('counts', ascending=False)
# df_merchant_category_by_fr_usr.head()

# df_state_by_fr_usr = df_trx_by_fr_usr.groupby('state').size().reset_index(name='counts').sort_values('counts', ascending=False)
# df_state_by_fr_usr.head()

# df_source_by_fr_usr = df_trx_by_fr_usr.groupby('source').size().reset_index(name='counts').sort_values('counts', ascending=False)
# df_source_by_fr_usr.head()


In [ ]:
# majority distribution of normal users trx
majority_distribution_of_normal_users_trx = """
with nr_usr as (select distinct(user_id), count(index) AS count from transactions where user_id not in (select fraudsters.user_id
from fraudsters) group by user_id order by count desc)
select user_id from nr_usr where count < 500;
"""
df_mj_nr_usr = pd.read_sql(majority_distribution_of_normal_users_trx, client_pg)
nr_usr_ids = df_mj_nr_usr['user_id'].values.tolist()
df_trx_by_nr_usr = df_all_trx[df_all_trx.user_id.isin(nr_usr_ids)]

df_entry_method_by_nr_usr = df_trx_by_nr_usr.groupby('entry_method').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_entry_method_by_nr_usr.head()

df_type_by_nr_usr = df_trx_by_nr_usr.groupby('type').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_type_by_nr_usr.head()

df_currency_by_nr_usr = df_trx_by_nr_usr.groupby('currency').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_currency_by_nr_usr.head()

df_merchant_country_by_nr_usr = df_trx_by_nr_usr.fillna('null').groupby('merchant_country').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_merchant_country_by_nr_usr.head()

df_merchant_category_by_nr_usr = df_trx_by_nr_usr.fillna('null').groupby('merchant_category').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_merchant_category_by_nr_usr.head()

df_state_by_nr_usr = df_trx_by_nr_usr.groupby('state').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_state_by_nr_usr.head()

df_source_by_nr_usr = df_trx_by_nr_usr.groupby('source').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_source_by_nr_usr.head()

In [ ]:
# source of nr and fr users barh
df_source_nr_fr_users = df_source_by_nr_usr.merge(df_source_by_fr_usr, left_on='source', right_on='source', suffixes=('_normal_users', '_fraudulent_users'))
df_source_nr_fr_users = df_source_nr_fr_users.set_index('source')
df_source_nr_fr_users = df_source_nr_fr_users.rename(columns={'counts_normal_users': 'normal users', 'counts_fraudulent_users': 'fraudulent users'})
df_source_nr_fr_users.plot.barh(title="count vs source of normal users transactions and fraudulent users transactions")

# # entry method of nr and fr users barh
df_entry_method_nr_fr_users = df_entry_method_by_nr_usr.merge(df_entry_method_by_fr_usr, left_on='entry_method', right_on='entry_method', suffixes=('_normal_users', '_fraudulent_users'))
df_entry_method_nr_fr_users.head()
df_entry_method_nr_fr_users = df_entry_method_nr_fr_users.set_index('entry_method')
df_entry_method_nr_fr_users = df_entry_method_nr_fr_users.rename(columns={'counts_normal_users': 'normal users', 'counts_fraudulent_users': 'fraudulent users'})
df_entry_method_nr_fr_users.plot.barh(title="count vs entry_method of normal users transactions and fraudulent users transactions")

# # type of nr and fr users barh
# df_type_nr_fr_users = df_type_by_nr_usr.merge(df_type_by_fr_usr, left_on='type', right_on='type', suffixes=('_normal_users', '_fraudulent_users'))
df_type_nr_fr_users.head()
df_type_nr_fr_users = df_type_nr_fr_users.set_index('type')
df_type_nr_fr_users = df_type_nr_fr_users.rename(columns={'counts_normal_users': 'normal users', 'counts_fraudulent_users': 'fraudulent users'})
df_type_nr_fr_users.plot.barh(title="count vs type of normal users transactions and fraudulent users transactions")

# # currency of nr and fr users barh
df_currency_nr_fr_users = df_currency_by_nr_usr.merge(df_currency_by_fr_usr, left_on='currency', right_on='currency', suffixes=('_normal_users', '_fraudulent_users'))
df_currency_nr_fr_users.head()
df_currency_nr_fr_users = df_currency_nr_fr_users.set_index('currency')
df_currency_nr_fr_users = df_currency_nr_fr_users.rename(columns={'counts_normal_users': 'normal users', 'counts_fraudulent_users': 'fraudulent users'})
df_currency_nr_fr_users.plot.barh(title="count vs currency of normal users transactions and fraudulent users transactions")

# # merchant country of nr and fr users barh
df_merchant_country_nr_fr_users = df_merchant_country_by_nr_usr.merge(df_merchant_country_by_fr_usr, left_on='merchant_country', right_on='merchant_country', suffixes=('_normal_users', '_fraudulent_users'))
df_merchant_country_nr_fr_users.head()
df_merchant_country_nr_fr_users = df_merchant_country_nr_fr_users.set_index('merchant_country')
df_merchant_country_nr_fr_users = df_merchant_country_nr_fr_users[df_merchant_country_nr_fr_users.counts_fraudulent_users > 10]
df_merchant_country_nr_fr_users = df_merchant_country_nr_fr_users.rename(columns={'counts_normal_users': 'normal users', 'counts_fraudulent_users': 'fraudulent users'})
df_merchant_country_nr_fr_users.plot.barh(title="count vs merchant_country of normal users transactions and fraudulent users transactions")

# # merchant category of nr and fr users barh
df_merchant_category_nr_fr_users = df_merchant_category_by_nr_usr.merge(df_merchant_category_by_fr_usr, left_on='merchant_category', right_on='merchant_category', suffixes=('_normal_users', '_fraudulent_users'))
df_merchant_category_nr_fr_users = df_merchant_category_nr_fr_users.set_index('merchant_category')
df_merchant_category_nr_fr_users = df_merchant_category_nr_fr_users[df_merchant_category_nr_fr_users.counts_fraudulent_users > 10]
df_merchant_category_nr_fr_users = df_merchant_category_nr_fr_users.rename(columns={'counts_normal_users': 'normal users', 'counts_fraudulent_users': 'fraudulent users'})
df_merchant_category_nr_fr_users.plot.barh(title="count vs merchant_category of normal users transactions and fraudulent users transactions")

# # state of nr and fr users barh
df_state_nr_fr_users = df_state_by_nr_usr.merge(df_state_by_fr_usr, left_on='state', right_on='state', suffixes=('_normal_users', '_fraudulent_users'))
df_state_nr_fr_users.head()
df_state_nr_fr_users = df_state_nr_fr_users.set_index('state')
df_state_nr_fr_users = df_state_nr_fr_users.rename(columns={'counts_normal_users': 'normal users', 'counts_fraudulent_users': 'fraudulent users'})
df_state_nr_fr_users.plot.barh(title="count vs state of normal users transactions and fraudulent users transactions")

# pie chart entry_method of fr and nr usr
df_entry_method_by_nr_usr.set_index('entry_method').plot.pie(y="counts", autopct='%1.0f%%', title="normal users")
df_entry_method_by_fr_usr.set_index('entry_method').plot.pie(y="counts", autopct='%1.0f%%', title="fraudulent users")

# pie chart entry_method of fr and nr usr
df_entry_method_by_nr_usr.set_index('entry_method').plot.pie(y="counts", autopct='%1.0f%%', title="normal users")
df_entry_method_by_fr_usr.set_index('entry_method').plot.pie(y="counts", autopct='%1.0f%%', title="fraudulent users")



In [ ]:
# finding consecutive transaction of normal users

df_trx_by_nr_usr_processed = df_trx_by_nr_usr;
df_trx_by_nr_usr_processed = df_trx_by_nr_usr_processed.set_index('user_id')
df_trx_by_nr_usr_processed = df_trx_by_nr_usr_processed.fillna('null')
list_3 = []

def iterator(user_id, current_trxs_of_user_id):
    list_iterator = [user_id]
    current_amount = None
    current_occurence = 0
    for trx in current_trxs_of_user_id: 
        if trx[0] == 'CARD_PAYMENT': 
            if current_amount == None:
                current_amount = trx[1]
            elif trx[1] == current_amount:
                current_occurence += 1
            elif trx[1] != current_amount:
                if current_occurence != 0: 
                    list_iterator.append(current_occurence)
                    list_3.append(list_iterator)
                    list_iterator = [user_id]
                current_amount = trx[1]
                current_occurence = 0
        else:
            if current_occurence > 0:
                list_iterator.append(current_occurence)
                list_3.append(list_iterator)
                list_iterator = [user_id]
            current_amount = trx[1]
            current_occurence = 0
    if current_occurence > 0:
        list_iterator.append(current_occurence)
        list_3.append(list_iterator)
        list_iterator = [user_id]


for user_id in nr_usr_ids:
    current_trxs_of_user_id = df_trx_by_nr_usr_processed.loc[user_id, ['type', 'amount']].values
    if(isinstance(current_trxs_of_user_id[0], str) == False):
        iterator(user_id, current_trxs_of_user_id)

df_n_3 = pd.DataFrame(list_3, columns=['user_id', 'num_cont_trx'])
df_n_3.head()
df_n_3_grouped = df_n_3.groupby('num_cont_trx').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_n_3_grouped['user_type'] = 'normal_users'
df_n_3_grouped.head()

In [ ]:
# finding consecutive transctions of fraudulents users

df_trx_by_fr_usr_processed = df_trx_by_fr_usr;
df_trx_by_fr_usr_processed = df_trx_by_fr_usr_processed.set_index('user_id')
df_trx_by_fr_usr_processed = df_trx_by_fr_usr_processed.fillna('null')
list_4 = []

def iterator(user_id, current_trxs_of_user_id):
    list_iterator = [user_id]
    current_amount = None
    current_occurence = 0
    for trx in current_trxs_of_user_id:
        print(trx, user_id)      
        if trx[0] == 'CARD_PAYMENT': 
            if current_amount == None:
                current_amount = trx[1]
            elif trx[1] == current_amount:
                current_occurence += 1
            elif trx[1] != current_amount:
                if current_occurence != 0:
                    list_iterator.append(current_occurence)
                    list_4.append(list_iterator)
                    list_iterator = [user_id]
                current_amount = trx[1]
                current_occurence = 0
        else:
            if current_occurence > 0:
                list_iterator.append(current_occurence)
                list_4.append(list_iterator)
                list_iterator = [user_id]
            current_amount = trx[1]
            current_occurence = 0
    if current_occurence > 0:
        list_iterator.append(current_occurence)
        list_4.append(list_iterator)
        list_iterator = [user_id]


for user_id in fr_usr_ids:
    current_trxs_of_user_id = df_trx_by_fr_usr_processed.loc[user_id, ['type', 'amount']].values
    if(isinstance(current_trxs_of_user_id[0], str) == False):
        iterator(user_id, current_trxs_of_user_id)

df_f_4 = pd.DataFrame(list_4, columns=['user_id', 'num_cont_trx'])
df_f_4.tail()
df_f_4_grouped = df_f_4.groupby('num_cont_trx').size().reset_index(name='counts').sort_values('counts', ascending=False)
df_f_4_grouped['user_type'] = 'fraudulent users'
df_f_4_grouped.head()

In [ ]:
# create categorical scatter plot

df_n_with_f_consecutive_card_trx = pd.concat([df_n_3_grouped, df_f_4_grouped], ignore_index=True)
df_n_with_f_consecutive_card_trx = df_n_with_f_consecutive_card_trx.sort_values('num_cont_trx')

sns.catplot(x="num_cont_trx", y="counts", kind="swarm", data=df_n_with_f_consecutive_card_trx, hue="user_type", height=5, aspect=3);

In [ ]:
# find out user characteristics, noticeably more than 90% are from Great Briatin. And also more than 90% of them has a exactly the same phone_country which is GB||JE||IM||GG

users_fraudsters = """
select * from users where id in (select fraudsters.user_id from fraudsters);
"""

df_users_fraudsters = pd.read_sql(users_fraudsters, client_pg)
df_list_phone_country_frausters = df_users_fraudsters.groupby('phone_country').size().reset_index(name='counts')
df_list_phone_country_frausters = df_list_phone_country_frausters.set_index('phone_country')
df_list_phone_country_frausters.plot.pie(y="counts", autopct='%1.0f%%', frame=True, radius=0.5)

In [ ]:
# users who've made non card payment and non manu transactions
query_users_w_trx_not_cp_misc_manu = """
select distinct(user_id) from transactions where type != 'CARD_PAYMENT' and (entry_method != 'manu')
"""
# users whose phone_country is 'GB||JE||IM||GG'
query_usr_w_pc = """
select * from users where phone_country = 'GB||JE||IM||GG';
"""
df_usr_non_cp = pd.read_sql(query_users_w_trx_not_cp_misc_manu, client_pg)
df_usr_w_pc = pd.read_sql(query_usr_w_pc, client_pg)
list_usr_cp = df_usr_non_cp['user_id'].values
list_usr_pc = df_usr_w_pc['id'].values
df_n_3_cp_misc_manu = df_n_3[(df_n_3['num_cont_trx'] <= 15) & (df_n_3['num_cont_trx'] >= 7) & ~df_n_3.user_id.isin(list_usr_cp) & df_n_3.user_id.isin(list_usr_pc)]

# possible new fraudsters
print(df_n_3_cp_misc_manu.user_id.unique())